## Week 8: Merging, Grouping, Plotting

### March 06, 2024

### Krishnapriya Vishnubhotla

## Recap

Last week, we learnt about the `pandas` library, and how to use it to:
- Load CSV files
    - DataFrame.shape
    - DataFrame.head(), DataFrame.tail()
- Clean the data
    - Rename columns (with DataFrame.rename)
    - Convert columns to the correct data types (DataFrame.convert_dtypes())
    - Replace missing values (with pd.NA)
    - Filter subsets (both columns and rows)
        - Boolean Filtering

- Transform the data
    - Operations on a single numerical column (multiply, round on a Series object)
    - Operations with multiple columns (add, multiply)
- Summarize the data
    - DataFrame.describe(), Series.describe()
    - Series.sum(), Series.mean()
    - Series.unique(), Series.nunique(), Series.value_counts()

## Read our data file

We will continue working with the `PanTHERIA` dataset from last lecture

In [ ]:
# import the library
import pandas as pd

In [ ]:
species_raw_data = pd.read_csv('PanTHERIA_WR05_Aug2008.csv')
display(species_raw_data)

## Basic data cleaning
Let us replicate the data cleaning operations from last time:
- Filter to important columns
- Rename columns
- Convert data types
- Standardize missing values

In [ ]:
# Extract columns
important_columns = [
    "MSW05_Order",
    "MSW05_Binomial",
    "5-1_AdultBodyMass_g",
    "23-1_SexualMaturityAge_d",
    "14-1_InterbirthInterval_d",
    "17-1_MaxLongevity_m",
    "15-1_LitterSize"
]

species_subset_data = species_raw_data[important_columns]

In [ ]:
species_subset_data.head()

In [ ]:
# Rename columns
old_to_new = {
    "MSW05_Order": "Order",
    "MSW05_Binomial": "Binomial",
    "5-1_AdultBodyMass_g": "Mass (g)",
    "23-1_SexualMaturityAge_d": "Maturity (days)",
    "14-1_InterbirthInterval_d": "Interbirth (days)",
    "17-1_MaxLongevity_m": "Longevity (months)",
    "15-1_LitterSize": "Litter Size"
}
species_renamed_data = species_subset_data.rename(columns=old_to_new)

In [ ]:
species_renamed_data.head()

In [ ]:
# Convert column types
species_converted_data = species_renamed_data.convert_dtypes()
species_converted_data.info()

In [ ]:
# Replace -999 values
species_data = species_converted_data.replace(-999, pd.NA)
species_data

In [ ]:
# a neat new function
species_data.sort_values(by='Longevity (months)')

In [ ]:
species_data.sort_values(by='Longevity (months)', ascending=False)

In [ ]:
# also applies to Series
species_data['Litter Size'].sort_values()

In [ ]:
# how can we remove NA?
is_na = species_data['Litter Size'].isna() 
# NOTE:  species_data['Litter Size'] == pd.NA will not work (try it out!)
# NOTE:  species_data['Litter Size'] == <NA> will not work (try it out!)
is_na

In [ ]:
# boolean filtering
species_data[is_na]

In [ ]:
# we want the rows where Litter Size is NOT null
# flip the sign!
~is_na

# NOTE:  species_data['Litter Size'] != pd.NA will not work (try it out!)
# NOTE:  species_data['Litter Size'] != <NA> will not work (try it out!)

In [ ]:
# boolean filtering!
species_data[~is_na]

In [ ]:
species_data[~is_na]['Litter Size'].sort_values()
# notice how dataframe operations can be chained

## Data Science Question of the Day

Are mammals that take longer go grow up (i.e, higher maturity age) at greater risk of extinction (IUCN status)?

We have maturity age information for mammals in the `species_data['Maturity (days)]` column.

Information about the Extinction level code is in a different data file! 

Let us read in `IUCN_status.csv` into a new DataFrame variable.

In [ ]:
iucn_raw_data = pd.read_csv("iucn_status.csv")
iucn_data = iucn_raw_data.convert_dtypes()
iucn_data.head()

## Merging Dataframes

We often will want to combine information from two (or more) data files to answer a data science question.

**Merging** is a `pandas` operation that *combines* the columns of two (or more) dataframes into a single mega-dataframe. 

How does `pandas` know which rows in each dataframe to merge together?
- We will tell `pandas` to merge two dataframes by **matching on a specific column** that is present in both datasets. This column can be thought of as a lookup key to merge the information from both dataframes

In [ ]:
species_data.head()

In [ ]:
iucn_data.head()

## Data Transformations on String Columns

We know we want to match the `iucn_data` and `species_data` dataframes on the column containing the name of the species.

BUT, the format of the species name in both these dataframes in slightly different....

- Camelus dromedarius (`species_data['Binomial`]) <==> Camelus_dromedarius (`iucn_data['species]`)


`pandas` looks for an exact match of the column values. 

Let us transform `iucn_data['species']` to have the same format as `species_data['Binomial']`.
- We want to *replace* the "_" in `iucn_data['species']` with a space.

In [ ]:
# recall from pre-midterm python
iucn_species_str = "Camelus_dromedarius"
iucn_species_str.replace("_", " ")

We can apply the same operation to all the values in the column (Series) in a single step (no `for` loops!)
- recall we performed these transformations on numerical columns last lecture

In [ ]:
# numerical
species_data['Maturity (days)']

In [ ]:
maturity_in_years = species_data['Maturity (days)'] / 365
display(maturity_in_years)

With `string` columns, we need to use a slightly different format.

In [ ]:
iucn_data['species']

In [ ]:
iucn_data['species'].str.upper()
# NOTE the additional ".str" before the transformation operation

In [ ]:
# now let us try the replace operation
iucn_data['species'].str.replace("_", " ")

In [ ]:
# tada! add this as a new column
iucn_data['species_formatted'] = iucn_data['species'].str.replace("_", " ")

In [ ]:
display(iucn_data)

## The Merge Operation

We can now ask `pandas` to combine the two dataframes by matching on:
- the `Binomial` column in `species_data`
- and the `species_formatted` column in `iucn_data`

In [ ]:
display(iucn_data['species_formatted'].head())
display(species_data['Binomial'].head())

Formally, we merge two `DataFrames` using a `pandas` function called `merge`.
We'll use `merge` with four arguments:

- `left`: the first `DataFrame` to merge
- `right`: the second `DataFrame` to merge
- `left_on`: the name of the column in the `left` `DataFrame` to match with
- `right_on`: the name of the column in the `right` `DataFrame` to match with

In [ ]:
combined_data = pd.merge(
    left=species_data,
    right=iucn_data,
    left_on="Binomial",
    right_on="species_formatted"
)

In [ ]:
display(combined_data)

In [ ]:
print(len(species_data.columns), len(iucn_data.columns), len(combined_data.columns))
print(len(species_data), len(iucn_data), len(combined_data))
# only the species present in both dataframes are in the combined dataframe

## Further cleaning

We have information about `order` in both dataframes: `species_data['Order']` and `iucn_data['order']`.

Do they always match?

In [ ]:
combined_data.head()

In [ ]:
# let us check if they match
is_order_match = combined_data['Order'] == combined_data['order']

In [ ]:
is_order_match

In [ ]:
is_order_match.value_counts()

In [ ]:
# let us filter out these rows!
# remember: Boolean filtering
combined_data[is_order_match]

In [ ]:
# check: where do they NOT match?
combined_data[~is_order_match]

In [ ]:
# Alternate method
is_order_mismatch = combined_data['Order'] != combined_data['order']
combined_data[is_order_mismatch]

In [ ]:
combined_data_clean = combined_data[is_order_match]

In [ ]:
display(combined_data_clean)
# optional: remove unnecessary columns
# keep_columns = [...]
# combined_data_clean = combined_data_clean[keep_columns]

## Recall our data science question

What is the relationship between maturity and extinction status?

Let us find out the **average maturity age for each extinction level** in the IUCN levels.

In [ ]:
combined_data_clean['iucn_status'].value_counts()

In [ ]:
combined_data_clean['Maturity (days)'].describe()

## Grouping

`pandas` has a `DataFrame.groupby()` function that allows us to compute aggregate measures for *grouped* subsets of the data in one (or two) line(s) of code.

The data can be *grouped* based on the values in a column. The *measures* can be any statistic about the other columns of the data:
- For each `order` (*group*), how many *unique* `species` exist?
- For each `species` (*group*), what is the *average* `Mass`?

Our question:
- For each `iucn_status` (*group*), what is the *average* `Maturity (days)`?

In [ ]:
# The long road #1
iucn_levels = list(combined_data_clean['iucn_status'].unique())
print(iucn_levels)

In [ ]:
# The long road #1

is_lc = combined_data_clean['iucn_status']=='LC' # boolean filter
lc_species = combined_data_clean[is_lc] # extract subset of the dataframe
lc_avg = lc_species['Maturity (days)'].mean() #compute mean of the maturity column values
print(lc_avg)

is_vu = combined_data_clean['iucn_status']=='VU'
vu_species = combined_data_clean[is_vu]
vu_avg = vu_species['Maturity (days)'].mean()
print(vu_avg)


# .... and so on for each IUCN level .....
# NOT efficient! Not enough automation!

In [ ]:
# The long read #2
avg_values = {}
for level in iucn_levels:
    is_level = combined_data_clean['iucn_status'] == level
    level_species = combined_data_clean[is_level]
    level_avg = level_species['Maturity (days)'].mean()
    avg_values[level] = level_avg

print(avg_values)

## better....but there is something even more better!

In [ ]:
# groupby and average

grouped_data = combined_data_clean.groupby('iucn_status')
group_avg = grouped_data['Maturity (days)'].mean()

In [ ]:
group_avg
# what is the type?

In [ ]:
group_avg['DD']

In [ ]:
# sort!
group_avg.round(3).sort_values()

# to answer the question, we need to *interpret* these values: do the more endangered species have 
# higher or lower average maturity?
# A job for you! (humans)

In [ ]:
# can be chained
combined_data_clean.groupby('iucn_status')['Maturity (days)'].mean()

## What are the data subsets for each group?

Recall the two steps we performed for groupby-aggregate:
1. `grouped_df = DataFrame.groupby(column1)`
2. `grouped_df[column2].agg_measure()`

Let us examine the output of step 1 in a bit more detail

In [ ]:
grouped_data = combined_data_clean.groupby('iucn_status')

In [ ]:
display(grouped_data)

This is an internal `pandas` representation that is not very accessible to us.

We can think of it as a dictionary that maps each group to the subset of the dataframe belonging to that group.

We can obtain the data subset for a particular group using the `DataFrameGroupBy.get_group()` method.

In [ ]:
grouped_data.get_group('CR')
# what is the type?

## One more example

Say we want to find out the *number of unique species* (measure) within each *order* (group) in our dataframe.

- we want to `groupby('order')`
- and for each group, find the `['Binomial'].nunique`

In [ ]:
combined_data_clean.head()

In [ ]:
# how many unique orders are there?
combined_data_clean['order'].nunique()

In [ ]:
# method 1: because each species is in a new row, we can simply use value_counts
combined_data_clean['order'].value_counts()

In [ ]:
# method 2: use groupby


In [ ]:
# DIY: a more complex example
# combined_data_clean.groupby('order')['iucn_status'].value_counts()
# combined_data_clean.groupby('order')['iucn_status'].value_counts()['Afrosoricida']
# combined_data_clean.groupby('order')['iucn_status'].value_counts()['Afrosoricida']['EN']

## Data Visualization

Finally, `pandas` offers some basic funtions to **plot** our data.

Visual representations can sometimes make certain relationships more apparent*, and can help us in choosing the right statistical tools to analyze our data.

*apparent != correct. Statistics is important.

## Plotting a Series
Let us plot the average Maturity values for each iucn level using the `group_avg` Series.

In [ ]:
# recall
group_avg

In [ ]:
# what kind of plot? A bar plot.
group_avg.plot.bar()

In [ ]:
# hmmm..let us sort it
# group_avg.sort_values().plot.bar()
group_avg.sort_values(ascending=False).plot.bar()

Each plotting method accepts optional arguments to configure the appearance of the plot.
For example:

- `title`: the title of the plot
- `xlabel`: the label for the x-axis
- `ylabel`: the label for the y-axis

In [ ]:
sorted_group_avg = group_avg.sort_values(ascending=False)
sorted_group_avg.plot.bar(
    title="Mean Age to Maturity by IUCN Status",
    xlabel="IUCN Status",
    ylabel="Mean Age to Maturity (days)"
)

By default, `Series.plot.bar` uses the `Series` *index* to label the bars.
We can customize these labels by using the `Series.rename(index=...)`.

In [ ]:
old_to_new = {
    "DD": "Data Deficient",
    "EN": "Endangered",
    "CR": "Critically Endangered",
    "VU": "Vulnerable",
    "NT": "Near Threatened",
    "LC": "Least Concern",
    "EW": "Extinct in the Wild",
    "EX": "Extinct"
}

group_maturities_renamed = sorted_group_avg.rename(index=old_to_new)


In [ ]:
group_maturities_renamed

In [ ]:
group_maturities_renamed.plot.bar(
    title="Mean Age to Maturity by IUCN Status",
    xlabel="IUCN Status",
    ylabel="Mean Age to Maturity (days)"
)

We can use the plotting functions with `DataFrame` objects as well! 
We need to specify the columns for the x and y axis.
For example:
- What is the relationship between maturity and litter size?

In [ ]:
combined_data_clean.plot.scatter(x='Litter Size', y='Maturity (days)', 
                                 title="Litter Size vs. Mean Age to Maturity Among Mammals")
# each point on the plot is a species (datapoint)

The distribution of a numerical list of values can be visualized as a *boxplot*.

In [ ]:
combined_data_clean.plot.box(
    column="Litter Size",
    title="Litter Size Among Mammals"
)

In [ ]:
# we can plot this distribution for each group, just with an extra argument!
combined_data_clean.plot.box(
    column="Litter Size",
    by='iucn_status',
    title="Litter Size Among Mammals by IUCN Status"
)

## Further reading

- [Pandas *Working with text data* guide](https://pandas.pydata.org/docs/user_guide/text.html)
- [Pandas *Chart visualization* guide](https://pandas.pydata.org/docs/user_guide/visualization.html)
